In [2]:
!pip install langchain_experimental

In [12]:
!pip install numexpr

In [15]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11756 sha256=06bb4ea2b1a1eaa5bd20c591cbf5e0571b436acbbee6fd8437410dd6f42a09dd
  Stored in directory: c:\users\huawei\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [38]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent, create_tool_calling_agent
from langchain.agents import AgentType, AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [10]:
llm = AzureChatOpenAI(
    api_key="                   ",
    azure_endpoint="              ",
    openai_api_version="                     ", 
    deployment_name="gpt-4o-mini",
    model_name="gpt-4o-mini",
    temperature=0.0
)

In [16]:
tools = load_tools(['llm-math', 'wikipedia'], llm=llm)

In [17]:
"""定义一个AgentExecutor类，这里使用initialize_agent方法，
比AgentExecutor更傻瓜一点，initialize_agent底层应该也是调用了AgentType，
但是做了一些预定义。
"""
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_27356\2802621003.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [19]:
type(agent)

langchain.agents.agent.AgentExecutor

In [26]:
#agent作为一个AgentExecutor对象似乎可以直接输入文本来执行，似乎是自动调用__call__
#但是之后就不知道可不可以了
response=agent("What's is the 25% of 300?")
print(response)



> Entering new AgentExecutor chain...
Thought: To find 25% of 300, I need to calculate 25/100 multiplied by 300. 
Action:
```
{
  "action": "Calculator",
  "action_input": "25/100 * 300"
}
```

Observation: Answer: 75.0
I now know the final answer.  
Final Answer: 75.0

> Finished chain.
{'input': "What's is the 25% of 300?", 'output': '75.0'}


In [20]:
query = "What's is the 25% of 300?"

In [22]:
#同时，用新增的invoke方法也可以进行执行，两者的效果一模一样
agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: To find 25% of 300, I can calculate it by multiplying 300 by 0.25. 
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```

Observation: Answer: 75.0
I now know the final answer.  
Final Answer: 75.0

> Finished chain.


{'input': "What's is the 25% of 300?", 'output': '75.0'}

In [42]:
#这是一种自己customize AgentExecutor的实现
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent1 = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent1, tools=tools, verbose=True)
agent_executor.invoke({"input":"1.1 and 1.08, which one is bigger?"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `1.1 > 1.08`


Answer: True1.1 is bigger than 1.08.

> Finished chain.


{'input': '1.1 and 1.08, which one is bigger?',
 'output': '1.1 is bigger than 1.08.'}

In [51]:
#测试一下wikipedia api，使用initialize_agent构建的AgentExecutor似乎不太可行
#但是直接使用AgentExecutor就可以

question = "Who is China's president?"
agent_executor.invoke({"input":question})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'President of China'}`


Page: President of China
Summary: The president of China, officially the president of the People's Republic of China, is the state representative of the People's Republic of China. On its own, it is a ceremonial office and has no real power in China's political system. While the office has many of the characteristics of a head of state, the Constitution of China does not define it as such. However, since 1993, the post has been held by the general secretary of the Chinese Communist Party (CCP) and chairman of the Central Military Commission, who is China's paramount leader.
The presidency is a part of the system of people's congress based on the principle of unified power in which the National People's Congress (NPC) functions as the only branch of government and as the highest state organ of power. The presidency is a state organ of the NPC and equivalent to, for instance, the State

{'input': "Who is China's president?",
 'output': "The current president of China is Xi Jinping. He has held the position since March 27, 1993, and also serves as the General Secretary of the Chinese Communist Party and Chairman of the Central Military Commission, making him China's paramount leader."}

In [56]:
#能进行python代码运行的agent
python_agent=create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)
type(python_agent) #这里可以看到create_python_agent同样也是定义了一个AgentExecutor

langchain.agents.agent.AgentExecutor

In [66]:
!pip install langchain_core

In [71]:
""" 如果希望使用langsmith进行调试，就需要设置以下环境变量
    !set LANGCHAIN_TRACING_V2=true
    !set LANGCHAIN_API_KEY=                    
"""
#当然也可以直接set_debug和set_verbose全局变量来打印详细执行过程
from langchain.globals import set_debug
set_debug(True)

python_agent.invoke("type(3.20)")
set_debug(False)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "type(3.20)"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "type(3.20)",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't know\" as the answer.\n\n\nPython_REPL - A Python shell. Use this to execute python commands. Input should b

开始自定义自己的tool

In [72]:
from langchain.agents import tool
from datetime import date

In [73]:
@tool
def time(text: str) -> str:
    """
    Returns todays date, use this for anyquestions related to knowing todays\
    date. The input should always be an empty string, and this function will\
    always return todays date - any date mathematics should occur outside\
    this function.
    """
    return str(date.today())

In [79]:
#基于AgentExecutor的标准实现
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent2 = create_tool_calling_agent(llm, [time], prompt)
date_agent = AgentExecutor(agent=agent2, tools=[time], verbose=True)
set_debug=True
date_agent.invoke({"input":"What day is it today?"})



> Entering new AgentExecutor chain...

Invoking: `time` with `{'text': ''}`


Today is March 2, 2025.

> Finished chain.


{'input': 'What day is it today?', 'output': 'Today is March 2, 2025.'}

In [80]:
date_agent1 = initialize_agent(
    [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [82]:
date_agent1({"input":"What day is it today? Output in numeric format."})



> Entering new AgentExecutor chain...
Thought: I need to find out today's date in numeric format. I will use the time tool to get the current date. 
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-03-02
I need to extract the numeric format of today's date from the observation. The date is in the format YYYY-MM-DD, which means today is March 2, 2025. 

Thought: I now know the final answer.
Final Answer: 2025-03-02

> Finished chain.


{'input': 'What day is it today? Output in numeric format.',
 'output': '2025-03-02'}